In [31]:
import pandas as pd
import requests

In [32]:
dates = {'begin': '2023-03-01', 'end': '2023-09-01'}
visits_request = requests.get('https://data-charts-api.hexlet.app/visits', params=dates)
visits_data = visits_request.json()
visits_df = pd.DataFrame(visits_data)

In [33]:
registrations_request = requests.get('https://data-charts-api.hexlet.app/registrations', params=dates)
registrations_data = registrations_request.json()
registrations_df = pd.DataFrame(registrations_data)

In [36]:
visits_df = visits_df[~visits_df['user_agent'].str.contains('bot')]
visits_unique = pd.DataFrame(visits_df.groupby('visit_id').agg('max')['datetime']).reset_index()
unique_visits = pd.merge(visits_df, visits_unique, on=['visit_id', 'datetime'])
unique_visits['date'] = pd.to_datetime(unique_visits['datetime']).dt.date
grouped_unique_visits = unique_visits.drop(['user_agent', 'datetime'], axis=1).groupby(['date', 'platform']).agg('count')
grouped_unique_visits.columns = ['visits']
registrations_df['date'] = pd.to_datetime(registrations_df['datetime']).dt.date
grouped_registrations = registrations_df.drop(['registration_type', 'email', 'datetime'], axis=1).groupby(['date', 'platform']).agg('count')
grouped_registrations.columns = ['registrations']
conversions = pd.merge(grouped_unique_visits, grouped_registrations, on=['date', 'platform'])
conversions['conversion'] = (conversions['registrations'] / conversions['visits'] ).round(3)
conversion = conversions.reset_index()
conversion.columns = ['date_group', 'platform', 'visits', 'registrations', 'conversion']
conversion = conversion.sort_values('date_group')
conversion.to_json('conversion.json')